<a href="https://colab.research.google.com/github/sneha-002/Brain-Stroke-Detection/blob/main/brainstroke_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from os import listdir
import tensorflow as tf


from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from keras.models import *
from keras.layers import *
from keras import *
from tensorflow import keras
from tensorflow.keras import layers
from PIL import Image, ImageOps
from keras.models import Model, load_model
from keras.layers import Input ,BatchNormalization , Activation
from keras.layers.convolutional import Conv2D, UpSampling2D
from keras.layers.pooling import MaxPooling2D
from keras import optimizers
from tensorflow.keras.utils import to_categorical
import keras.backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler

LOADING DATA

In [ ]:
!pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sowmyareddy09","key":"c68950e9ba387ba271ad613d97be364c"}'}

IMPORTING LIBRARIES

In [ ]:
! mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d rizwanulhoqueratul/brain-mri-data

In [ ]:
! unzip brain-mri-data.zip
! ls

In [ ]:
! ls Brain_Data_Organised

PREPROCESSING THE DATA

In [ ]:
def load_data(dir_list, image_size):
    x_train = []
    y_train= []
    img_width, img_height = image_size

    for directory in dir_list:
        for filename in listdir(directory):
            path = directory + '/' + filename
            image = cv2.imread(path, 0)
            image = cv2.resize(image, dsize=(img_width, img_height), interpolation=cv2.INTER_CUBIC)
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB )
            image = image / 255.0
            x_train.append(image)
            if directory[-6:] == 'Normal':
                y_train.append([0])
            else:
                y_train.append([1])

    x_train = np.array(x_train)
    y_train = np.array(y_train)

    # Shuffle the data
    x_train, y_train = shuffle(x_train, y_train)
    print(directory[-6:])
    print(f'Number of examples is: {len(x_train)}')
    print(f'X shape is: {x_train.shape}')
    print(f'y shape is: {y_train.shape}')

    return x_train, y_train

In [ ]:
path='/content/Brain_Data_Organised/Train/'
path_normal=path + 'Normal'
path_stroke=path + 'Stroke'
IMG_WIDTH, IMG_HEIGHT = (224, 224)

x_train, y_train = load_data([path_normal, path_stroke], (IMG_WIDTH, IMG_HEIGHT))

DATA SPLITING

In [ ]:
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=0.2)

In [ ]:
print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)
#print(y_train[0:100])
print(y_train[653])

In [ ]:
y_train = to_categorical(y_train,2)
y_val = to_categorical(y_val,2)

VISUALIZE THE DATA

In [ ]:
plt.imshow(x_train[0],cmap="gray")
plt.show()
print(x_train[0].shape)

BUILD THE MODEL

In [ ]:
model=Sequential()
model.add(ZeroPadding2D(input_shape=(224,224,3),padding=(1,1)))
model.add(Conv2D(filters=96,kernel_size=(7,7),strides=(2,2),padding='valid', activation='relu'))
model.add(MaxPooling2D(pool_size =(3, 3),strides=(2,2),padding='same'))
model.add(BatchNormalization(axis=-1))

model.add(Conv2D(filters=256,kernel_size=(5,5),strides=(2,2),padding='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same'))
model.add(BatchNormalization(axis=-1))

model.add(Conv2D(filters=384,kernel_size=(3,3),padding='same', activation='relu'))

model.add(Conv2D(filters=384,kernel_size=(3,3),padding='same', activation='relu'))

model.add(Conv2D(filters=256,kernel_size=(3,3),padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='valid'))

model.add(Flatten())
model.add(Dense(4096))
model.add(Dense(4096))
model.add(Dense(2))
model.add(Softmax(axis=-1))
model.summary()

INCEPTION

In [ ]:

from tensorflow.keras.applications import InceptionV3

inception = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

output_layer = tf.keras.layers.Dense(2, activation='softmax')

model = tf.keras.Sequential([inception, tf.keras.layers.GlobalAveragePooling2D(), output_layer])
model.summary()


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2

# Load the pre-trained MobileNetV2 model
mobilenet = MobileNetV2(weights='imagenet',include_top=False, input_shape=(224, 224, 3))

output_layer = tf.keras.layers.Dense(2, activation='softmax')

model = tf.keras.Sequential([mobilenet, tf.keras.layers.GlobalAveragePooling2D(), output_layer])
model.summary()
# Print the model summary
print(mobilenet.summary())


TRAIN THE MODEL

In [ ]:
opt = keras.optimizers.SGD(learning_rate=0.01)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#y_train=y_train.reshape(-1,)

In [ ]:
def get_callbacks(model_name):
    callbacks =[]
    tensor_board = tf.keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0)
    callbacks.append(tensor_board)
    checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=f'model.{model_name}.h5', verbose=1, monitor='val_loss',mode='min',save_best_only=True)
    callbacks.append(checkpoint)
    anne = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=2, min_lr=0.0000001,min_delta=0.00001,mode='auto')
    callbacks.append(anne)
    return callbacks

In [ ]:
callbacks = get_callbacks('model')
model.fit( x_train,y_train,batch_size=16,epochs=10,validation_data= (x_val,y_val),callbacks=[callbacks])

GRAPH

In [ ]:
history = model.history.history

def plot_metrics(history):

    train_loss = history['loss']
    val_loss = history['val_loss']
    train_acc = history['accuracy']
    val_acc = history['val_accuracy']

    # Loss
    plt.figure()
    plt.plot(train_loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.title('Loss')
    plt.legend()
    plt.show()

    # Accuracy
    plt.figure()
    plt.plot(train_acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.title('Accuracy')
    plt.legend()
    plt.show()

In [ ]:
plot_metrics(history)

TEST

In [ ]:
classes={ 0:'Normal',1:'Stroke'}

In [ ]:
#y=np.expand_dims(x_train[653],0)
x=plt.imread('/content/Brain_Data_Organised/Test/Normal/100 (21).jpg',0)
x=cv2.cvtColor(x, cv2.COLOR_GRAY2RGB )
x=cv2.resize(x, dsize=(224,224), interpolation=cv2.INTER_CUBIC)
x = x / 255.0
x=np.array(x)
plt.imshow(x)
plt.show()
y=np.expand_dims(x,0)
h= model.predict(y)
a=np.argmax(h)
print(classes[a])

ROUGH

In [ ]:
def net(img_shape):
  input = Input(img_shape)

  x1 = Conv2D(64, 7, strides=2, padding='same')(input)
  x2 = MaxPool2D(3, strides=2, padding='same')(x1)

  x = BatchNormalization()(x2)
  x = ReLU()(x)
  x = Conv2D(filters=128, kernel_size=(1,1), strides=(1,1), padding='same')(x)
  x = BatchNormalization()(x)
  x = ReLU()(x)
  x = Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), padding='same')(x)
  x3= Concatenate()([x2, x])

  x = BatchNormalization()(x3)
  x = ReLU()(x)
  x = Conv2D(filters=128, kernel_size=(1,1), strides=(1,1), padding='same')(x)
  x = BatchNormalization()(x)
  x = ReLU()(x)
  x = Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), padding='same')(x)
  x4= Concatenate()([x3, x])

  x = BatchNormalization()(x4)
  x = ReLU()(x)
  x = Conv2D(filters=128, kernel_size=(1,1), strides=(1,1), padding='same')(x)
  x = BatchNormalization()(x)
  x = ReLU()(x)
  x = Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), padding='same')(x)
  x5= Concatenate()([x4, x])

  x = BatchNormalization()(x5)
  x = ReLU()(x)
  x = Conv2D(filters=128, kernel_size=(1,1), strides=(1,1), padding='same')(x)
  x = BatchNormalization()(x)
  x = ReLU()(x)
  x = Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), padding='same')(x)
  x6= Concatenate()([x5, x])

  x = BatchNormalization()(x6)
  x = ReLU()(x)
  x = Conv2D(filters=128, kernel_size=(1,1), strides=(1,1), padding='same')(x)
  x = BatchNormalization()(x)
  x = ReLU()(x)
  x = Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), padding='same')(x)
  x7= Concatenate()([x6, x])

  x = BatchNormalization()(x7)
  x = ReLU()(x)
  x = Conv2D(filters=128, kernel_size=(1,1), strides=(1,1), padding='same')(x)
  x = BatchNormalization()(x)
  x = ReLU()(x)
  x = Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), padding='same')(x)
  x8= Concatenate()([x7, x])

  x = BatchNormalization()(x8)
  x = ReLU()(x)
  x = Conv2D(filters=32, kernel_size=(1,1), strides=(1,1), padding='same')(x)
  x9 = AvgPool2D(pool_size=(2,2),strides=(2,2), padding='same')(x)

  x = BatchNormalization()(x9)
  x = ReLU()(x)
  x = Conv2D(filters=128, kernel_size=(1,1), strides=(1,1), padding='same')(x)
  x = BatchNormalization()(x)
  x = ReLU()(x)
  x = Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), padding='same')(x)
  x10= Concatenate()([x9, x])

  x = BatchNormalization()(x10)
  x = ReLU()(x)
  x = Conv2D(filters=128, kernel_size=(1,1), strides=(1,1), padding='same')(x)
  x = BatchNormalization()(x)
  x = ReLU()(x)
  x = Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), padding='same')(x)
  x11= Concatenate()([x10, x])

  x = BatchNormalization()(x11)
  x = ReLU()(x)
  x = Conv2D(filters=128, kernel_size=(1,1), strides=(1,1), padding='same')(x)
  x = BatchNormalization()(x)
  x = ReLU()(x)
  x = Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), padding='same')(x)
  x12= Concatenate()([x11, x])

  x = BatchNormalization()(x12)
  x = ReLU()(x)
  x = Conv2D(filters=128, kernel_size=(1,1), strides=(1,1), padding='same')(x)
  x = BatchNormalization()(x)
  x = ReLU()(x)
  x = Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), padding='same')(x)
  x13= Concatenate()([x12, x])

  x = BatchNormalization()(x13)
  x = ReLU()(x)
  x = Conv2D(filters=128, kernel_size=(1,1), strides=(1,1), padding='same')(x)
  x = BatchNormalization()(x)
  x = ReLU()(x)
  x = Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), padding='same')(x)
  x14= Concatenate()([x13, x])

  x = BatchNormalization()(x14)
  x = ReLU()(x)
  x = Conv2D(filters=128, kernel_size=(1,1), strides=(1,1), padding='same')(x)
  x = BatchNormalization()(x)
  x = ReLU()(x)
  x = Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), padding='same')(x)
  x15= Concatenate()([x14, x])

  x = GlobalAvgPool2D()(x15)
  output = Dense(2,activation='softmax')(x)
  model = Model(input, output)
  return model
model=net((224,224,3))
model.summary()


In [ ]:
img = keras.preprocessing.image.load_img("/content/Brain_Data_Organised/Test/Normal/100 (21).jpg", target_size=(224,224,3))
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create batch axis

predictions = model.predict(img_array)
score = predictions[0]
print("This image is %.2f percent cat and %.2f percent dog." % (100 * (1 - score), 100 * score))

In [ ]:
def densenet(img_shape, n_classes, f=32):
  repetitions = 6, 12

  def bn_rl_conv(x, f, k=1, s=1, p='same'):
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(f, k, strides=s, padding=p)(x)
    return x


  def dense_block(tensor, r):
    for _ in range(r):
      x = bn_rl_conv(tensor, 4*f)
      x = bn_rl_conv(x, f, 3)
      tensor = Concatenate()([tensor, x])
    return tensor


  def transition_block(x):
    x = bn_rl_conv(x, K.int_shape(x)[-1] // 2)
    x = AvgPool2D(2, strides=2, padding='same')(x)
    return x


  input = Input(img_shape)

  x = Conv2D(64, 7, strides=2, padding='same')(input)
  x = MaxPool2D(3, strides=2, padding='same')(x)

  for r in repetitions:
    d = dense_block(x, r)
    x = transition_block(d)

  x = GlobalAvgPool2D()(d)

  output = Dense(n_classes, activation='softmax')(x)

  model = Model(input, output)
  return model
K.clear_session()
model=densenet((224,224,3),2,f=32)
model.summary()

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape = (224, 224, 3), activation = 'sigmoid'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(64, (3, 3), activation = 'sigmoid'))
model.add(Conv2D(32, (3, 3), activation = 'sigmoid'))
model.add(Conv2D(18, (3, 3), activation = 'sigmoid'))
model.add(Conv2D(16, (3, 3), activation = 'sigmoid'))
model.add(Conv2D(14, (3, 3), activation = 'tanh'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(128, (3, 3), activation = 'tanh'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(256, (3, 3), activation = 'tanh'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Flatten())
model.add(Dense(units = 150, activation = 'tanh'))
model.add(Dropout(0.25))
model.add(Dense(units = 2, activation = 'sigmoid'))

opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer = opt, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(x_train, y_train, batch_size=32, epochs=10,validation_data=(x_val, y_val),)

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",input_shape=(224, 224, 3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

In [ ]:
model=Sequential()
model.add(data_augmentation)
model.add(ZeroPadding2D(padding=(1,1)))
model.add(Conv2D(filters=96,kernel_size=(7,7),strides=(2,2),padding='valid', activation='sigmoid'))
model.add(MaxPooling2D(pool_size =(3, 3),strides=(2,2),padding='same'))
model.add(LayerNormalization(axis=-1))

model.add(Conv2D(filters=256,kernel_size=(5,5),strides=(2,2),padding='valid', activation='sigmoid'))
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same'))
model.add(LayerNormalization(axis=-1))

model.add(Conv2D(filters=384,kernel_size=(3,3),padding='same', activation='sigmoid'))

model.add(Conv2D(filters=384,kernel_size=(3,3),padding='same', activation='relu'))

model.add(Conv2D(filters=256,kernel_size=(3,3),padding='same', activation='sigmoid'))
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='valid'))

model.add(Flatten())
model.add(Dense(4096))
model.add(Dropout(0.2))
model.add(Dense(4096))
model.add(Dense(2))
model.add(Softmax(axis=-1))

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
    ]
)

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
        layers.experimental.preprocessing.RandomZoom(height_factor=(0.2, 0.3), width_factor=None, fill_mode='reflect',
        interpolation='bilinear', seed=None, fill_value=0.0),
                layers.experimental.preprocessing.RandomCrop(height= 224, width = 224, seed=None),
    ]
)

In [ ]:
OPTIMIZER =Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
base_model = tf.keras.applications.DenseNet121(input_shape=IMG_SHAPE,include_top=False,weights='imagenet')
#base_model = tf.keras.applications.VGG16(input_shape=IMG_SHAPE,include_top=False,weights='imagenet')
base_model.trainable = True
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(1, activation='sigmoid',name='Final')(x)
model = Model(inputs=base_model.input, outputs=predictions)
#model.load_weights(dire)
model.compile(loss ='binary_crossentropy',optimizer=OPTIMIZER,metrics=METRICS)

In [ ]:
height=224
width=224
channels=3
def network(input_size=(224,224,3)):
   inputs=Input(input_size)
   x1=data_augmentation(inputs)
   x1=ZeroPadding2D(input_shape=(224,224,3),padding=(1,1))(x1)
   c1=Conv2D(filters=96,kernel_size=(7,7),strides=(2,2),padding='valid', activation='sigmoid')(x1)
   p1=MaxPooling2D(pool_size =(3, 3),strides=(2,2),padding='same')(c1)
   n1=LayerNormalization(axis=-1)(p1)

   c2=Conv2D(filters=256,kernel_size=(5,5),strides=(2,2),padding='valid', activation='sigmoid')(n1)
   p2=MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(c2)
   n2=LayerNormalization(axis=-1)(p2)

   c3=Conv2D(filters=384,kernel_size=(3,3),padding='same', activation='relu')(n2)

   c4=Conv2D(filters=384,kernel_size=(3,3),padding='same', activation='sigmoid')(c3)

   c5=Conv2D(filters=256,kernel_size=(3,3),padding='same', activation='sigmoid')(c4)
   p3=MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='valid')(c5)


   x3=ZeroPadding2D(input_shape=(224,224,3),padding=(2,2))(inputs)
   c6=Conv2D(filters=96,kernel_size=(11,11),strides=(4,4),padding='valid', activation='sigmoid')(x3)
   p4=MaxPooling2D(pool_size =(3, 3),strides=(2,2),padding='valid')(c6)
   n3=LayerNormalization(axis=-1)(p4)

   c7=Conv2D(filters=256,kernel_size=(5,5),strides=(1,1),padding='same', activation='sigmoid')(n3)
   p5=MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='valid')(c7)
   n4=LayerNormalization(axis=-1)(p5)

   c8=Conv2D(filters=384,kernel_size=(3,3),padding='same', activation='relu')(n4)

   c9=Conv2D(filters=384,kernel_size=(3,3),padding='same', activation='sigmoid')(c8)

   c10=Conv2D(filters=256,kernel_size=(3,3),padding='same', activation='sigmoid')(c9)
   p6=MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='valid')(c10)
   concat=Concatenate()([p3,p6])
   #c11=c10=Conv2D(filters=1,kernel_size=(1,1),padding='same', activation='relu')(concat)

   x2=Flatten()(concat)
   x2=Dropout(0.5)(x2)
   x2=Dense(4096)(x2)
   x2=Dropout(0.5)(x2)
   x2=Dense(4096)(x2)
   x2=Dense(2,activation="softmax")(x2)
   return Model(inputs=[inputs],outputs=[x2])
model=network()
model.summary()